## Using Converse API

## Prerequisites

Run the cells in this section to install the required packages. ⚠️ You may see pip dependency errors, you can safely ignore these errors. ⚠️

_IGNORE ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts._

In [ ]:
%pip install -q --force-reinstall \
    "botocore>=1.40.26" \
    "awscli>=1.29.57" \
    "boto3>=1.40.26"

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

The following examples show how to interact with Nova via the Converse API ([API Docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html))

In [ ]:
import boto3
import json
import base64
from botocore.client import Config
from pprint import pprint
import textwrap
from pprint import pprint

LITE_MODEL_ID = "us.amazon.nova-2-lite-v1:0"
modelId = LITE_MODEL_ID
# Create a Bedrock Runtime client
client = boto3.client("bedrock-runtime", 
                      region_name="us-east-1", 
                      config=Config(read_timeout=10000))


In [ ]:
 ## Functions to converse with Nova models
def converse_with_nova(client, native_request, show_full_response=False):
    # Invoke the model and extract the response body.
    model_response = client.converse(**native_request)

    # change the show_full_response flag to True to see full response
    if show_full_response:
        print("\n[Full Response]")
        pprint(model_response, indent=2)

    print("\n[Response Content Text]")
    pprint(model_response["output"]["message"]["content"], indent=2, width=80)
    return None

In [ ]:
## Functions to converse with Nova models with streaming option

def converse_with_nova_with_streaming(client, native_request):

    # Invoke the model and extract the response body.
    model_response = client.converse_stream(**native_request)
    # change the show_full_response flag to True to see full response
    # Process the response stream
    stream = model_response.get("stream")
    if stream:
        for event in stream:
            if 'messageStart' in event:
                pass
            elif 'contentBlockDelta' in event:
                content_block_delta = event["contentBlockDelta"]
                if content_block_delta:
                    print(content_block_delta.get("delta").get("text"), end="")
    else:
        print("No response stream received.")


## **Converse with Nova Model**

In [ ]:
 # Define your system prompt(s).
system = [
    {
        "text": "You are an experienced publisher. For each user topic, respond with 3 potential book titles"
    }
]

# Your user prompt
messages = [
    {"role": "user", "content": [{"text": "a child graduating from high school"}]},
]

inf_params = {"maxTokens": 1000, "topP": 0.9, "temperature": 0.7} 

native_request = {
    "modelId": modelId,
    "system": system,
    "messages": messages,
    
    "inferenceConfig": inf_params,
    # define reasoning configuration 
    "additionalModelRequestFields": {
        "reasoningConfig": {
        "type": "enabled",
        "maxReasoningEffort": "low"
        }
    }
}

converse_with_nova(client, native_request, show_full_response=True)


### **Single Turn with Streaming Option**

In [ ]:
# Define your system prompt(s).
system = [
    {
        "text": "You are an experienced publisher. For each user topic, respond with 3 potential book titles"
    }
]

# Your user prompt
messages = [
    {"role": "user", "content": [{"text": "a child graduating from high school"}]},
]

inf_params = {
                "maxTokens": 1000, 
                "topP": 0.9, 
                "temperature": 0.7,                     
            } 

native_request = {
    "modelId": modelId,
    "messages": messages,
    "system": system,
    "inferenceConfig": inf_params,
    "additionalModelRequestFields": {
        "reasoningConfig": {
        "type": "enabled",
        "maxReasoningEffort": "low"
        }
    }
}

converse_with_nova_with_streaming(client, native_request)



### **Multi Turn with Streaming Option**

In [ ]:
# Define one or more messages using the "user" and "assistant" roles.
# Define your system prompt(s).
system = [
    {
        "text": "You are an helpful assistant. Please answer question from the contents in the message only. Do not rely on any external source."
    }
]

messages = [
    {"role": "user", "content": [{"text": "How many days are in a week?"}]},
    {"role": "assistant", "content": [{"text": "There are seven days in a week and first day of the week is Monday and last day is Sunday"}]},
    {"role": "user", "content": [{"text": "Which day is the first?"}]},
]

# Configure the inference parameters 
inf_params = {
        "maxTokens": 512, 
        "topP": 0.9, 
        "temperature": 0.1,
}



native_request = {
    "modelId": modelId,
    "messages": messages,
    "system": system,
    "inferenceConfig": inf_params,
    "additionalModelRequestFields": {
        "reasoningConfig": {
        "type": "enabled",
        "maxReasoningEffort": "low"
        }
    }
}

converse_with_nova_with_streaming(client, native_request)

### **Image Understanding**

In [ ]:

with open("media/kites_and_plane.png", "rb") as f:
    image = f.read()

messages = [
    {
        "role": "user",
        "content": [
            {"image": {"format": "png", "source": {"bytes": image}}},
            {"text": "Describe the following image"},
        ],
    }
]

# Configure the inference parameters 
inf_params = {"maxTokens": 1000, "topP": 0.9, "temperature": 0.7} 

native_request = {
    "modelId": modelId,
    "messages": messages,
    "system": system,
    "inferenceConfig": inf_params,
    # define reasoning configuration 
    "additionalModelRequestFields": {
        "reasoningConfig": {
        "type": "enabled",
        "maxReasoningEffort": "low"
        }
    }
}

converse_with_nova(client, native_request, show_full_response=False)


In [ ]:

# Open the image you'd like to use and encode it as a Base64 string.
with open("media/dog.jpeg", "rb") as image_file:
    dog_binary_data = image_file.read()

with open("media/cat.jpeg", "rb") as image_file:
    cat_binary_data = image_file.read()

messages = [
    {
        "role": "user",
        "content": [
            {"image": {"format": "jpeg", "source": {"bytes": dog_binary_data}}},
            {"image": {"format": "jpeg", "source": {"bytes": cat_binary_data}}},
            {"text": "What do these two images have in common"},
        ],
    }
]

# Configure the inference parameters 
inf_params = {"maxTokens": 1000, "topP": 0.9, "temperature": 0.7} 

native_request = {
    "modelId": modelId,
    "messages": messages,
    "system": system,
    "inferenceConfig": inf_params,
    # define reasoning configuration 
    "additionalModelRequestFields": {
        "reasoningConfig": {
        "type": "enabled",
        "maxReasoningEffort": "low"
        }
    }
}
converse_with_nova(client, native_request, show_full_response=False)

### S3 Path for Image Understanding

Replace the S3 URI with a valid S3 URI where images are located.

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "image": {
                    "format": "jpeg",
                    "source": {
                        "s3Location": {
                            #Replace the s3 bucket URI 
                            "uri": "s3://s3-demo-bucket-nova-2/cat2.jpeg"
                        }
                    },
                }
            },
            {"text": "Describe the following image"},
        ],
    }
]

# Configure the inference parameters 
inf_params = {"maxTokens": 1000, "topP": 0.9, "temperature": 0.7} 
## for Nova Lite 1.5 Model
native_request = {
    "modelId": modelId,
    "messages": messages,
    "system": system,
    "inferenceConfig": inf_params,
    # define reasoning configuration for Nova Lite 1.5 model
    "additionalModelRequestFields": {
        "reasoningConfig": {
        "type": "enabled",
        "maxReasoningEffort": "low"
        }
    }
}

converse_with_nova(client, native_request, show_full_response=False)

### **Video Understanding**


In [ ]:

with open("./media/the-sea.mp4", "rb") as file:
    media_bytes = file.read()
    media_base64 = base64.b64encode(media_bytes)


messages = [
    {
        "role": "user",
        "content": [
            {"video": {"format": "mp4", "source": {"bytes": media_bytes}}},
            {"text": "Describe the following video"},
        ],
    }
]

# Configure the inference parameters 
inf_params = {"maxTokens": 1000, "topP": 0.9, "temperature": 0.7} 
native_request = {
    "modelId": modelId,
    "messages": messages,
    "system": system,
    "inferenceConfig": inf_params,
    # define reasoning configuration 
    "additionalModelRequestFields": {
        "reasoningConfig": {
        "type": "enabled",
        "maxReasoningEffort": "low"
        }
    }
}

converse_with_nova(client, native_request, show_full_response=False)


### S3 Path for Video Understanding 

##### Replace the S3 bucket URI with URI where the video file is located

In [ ]:

messages = [
    {
        "role": "user",
        "content": [
            {
                "video": {
                    "format": "mp4",
                    "source": {
                        "s3Location": {
                            #Replace the s3 bucket URI 
                            "uri": "s3://s3-demo-bucket-nova-2/the-sea.mp4"
                        }
                    },
                }
            },
            {"text": "Describe the following video"},
        ],
    }
]


# Configure the inference parameters 
inf_params = {"maxTokens": 1000, "topP": 0.9, "temperature": 0.7} 
## for Nova Lite 1.5 Model
native_request = {
    "modelId": modelId,
    "messages": messages,
    "system": system,
    "inferenceConfig": inf_params,
    # define reasoning configuration for Nova Lite 1.5 model
    "additionalModelRequestFields": {
        "reasoningConfig": {
        "type": "enabled",
        "maxReasoningEffort": "low"
        }
    }
}

converse_with_nova(client, native_request, show_full_response=False)